importer données rotten tomatoes

In [236]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch import optim

import preprocesser
import outils
import modules
    
phraseID, sentenceID, sentences, sentiment = outils.load_tsv("../train.tsv")
train_indexes, test_indexes = outils.split_train_test(phraseID)

Module de preprocessing

Fonctions pour la gestion des inputs 

In [237]:
#lang = preprocesser.Preprocesser(sentences)
#lang.normalize()
#lang.addSentences()
#lang.save("preprocessing_IMDB")


lang = preprocesser.Preprocesser({})
lang.load("../../preprocessing_IMDB")

Fonctions pour l'apprentissage

In [254]:
def X(p,g,i,temperature):
    return np.exp((p[i]+g[i])/temperature)

#size parameters
embedding_size = 512
d = 200
dprime = 20
#modules
embedder = nn.Embedding(lang.n_words,embedding_size)
selector = modules.Selector(embedding_size+d)
skim_rnn = modules.RNN(embedding_size,dprime,2)
rnn = modules.RNN(embedding_size,d,2)
#loss criteria and optimizers
criterion = nn.NLLLoss()
test_criterion = nn.L1Loss()
optim_selector = optim.SGD(selector.parameters(), lr=1e-2)
optim_rnn = optim.SGD([{'params':skim_rnn.parameters()},{'params':rnn.parameters()}], lr=1e-2)
#other variables
skimcount,count = 0,0
temperature, gamma = 0.5,1
logp_skim = []
global_losses = []


#training snippet
for i in train_indexes[:200]:
    loss, selector_loss= 0,0
    #make the input and the target
    sentence, target = lang.corpus[i], sentiment[i]
    input, target = preprocesser.makeInputTarget(lang,sentence,target)
    input_size = input.size()[0]
    
    #go through each word
    for word in range(input_size):
        optim_selector.zero_grad(), optim_rnn.zero_grad()
        
        x, hidden = Variable(input[word]), rnn.initHidden()
        embedding = embedder(x).view(1,1,-1)
        #go through the selector
        x = torch.cat((embedding, hidden),2).view(1,-1)
        p = selector(x)
        logp_skim.append(p.data[0,1])     #skimming proba, for regularization purposes
        q = torch.multinomial(p.exp())
        choice = int(q.data[0,0])
        
        if choice == 0:
            #go through the normal RNN
            output, hidden = rnn(embedding, hidden)
        
        else:
            #go through the skim rnn, which implies 'cutting' the hidden state,
            #running through the neural network, and building the new hidden state.
            h0 = hidden.view(-1)[:dprime]
            output, h0 = skim_rnn(embedding, h0.view(1,1,-1))
            h1 = hidden.view(-1)[dprime:]
            hidden = torch.cat( (h0.view(1,1,-1), h1.view(1,1,-1)), 2)
            skimcount +=1
        count += 1
        
        #add to the current losses
        loss += criterion(output,target)
        r_word = torch.FloatTensor([outils.r(p, outils.gumbel(), temperature)[0].prod()])
        #selector_loss += p.view(-1)[choice]*Variable(loss.data * r_word)
        g = outils.gumbel()
        if choice==0:
            selector_loss += loss.data*( - X(p.view(-1).data,g,0,temperature) / ( (temperature*np.exp(p.view(-1).data[1]))*(X(p.view(-1).data,g,0,temperature) + X(p.view(-1).data,g,1,temperature))) )
        else:
            selector_loss += loss.data*( - X(p.view(-1).data,g,1,temperature) / ( (temperature*np.exp(p.view(-1).data[0]))*(X(p.view(-1).data,g,0,temperature) + X(p.view(-1).data,g,1,temperature))) )
        
    #final losses calculations, backwards, and gradient steps
    loss.backward(retain_graph=True)
    selector_loss = selector_loss + (gamma/input_size) * np.sum(logp_skim)
    #selector_loss.backward()
    
    selector.linear.weight.data -= selector_loss*1e-2
    optim_rnn.step()
    #optim_selector.step()
    
    
    
    
    if epoch%mod == 0:
        global_losses.append((loss.data[0], selector_loss))
        accs.append(accuracy())
    
    
    '''debug prints
    print '//////////////////'
    test = selector.linear.weight
    print selector.linear.weight
    print skim_rnn.h20.weight
    '''


-11.4325
[torch.FloatTensor of size 1]


-16.5351
[torch.FloatTensor of size 1]


-20.6538
[torch.FloatTensor of size 1]


-18.9012
[torch.FloatTensor of size 1]


-32.3169
[torch.FloatTensor of size 1]


-913.0001
[torch.FloatTensor of size 1]


-522.5339
[torch.FloatTensor of size 1]


-2.1652
[torch.FloatTensor of size 1]


-30.6347
[torch.FloatTensor of size 1]


-4.6262
[torch.FloatTensor of size 1]


-1.1638
[torch.FloatTensor of size 1]


-11.7915
[torch.FloatTensor of size 1]


-85.8136
[torch.FloatTensor of size 1]


-160.4016
[torch.FloatTensor of size 1]


-18.6583
[torch.FloatTensor of size 1]


-7.3473
[torch.FloatTensor of size 1]


-1115.2153
[torch.FloatTensor of size 1]


-8.4648
[torch.FloatTensor of size 1]


-143.5441
[torch.FloatTensor of size 1]


-3.4231
[torch.FloatTensor of size 1]


-9.7909
[torch.FloatTensor of size 1]


-11.8245
[torch.FloatTensor of size 1]


-5.4501
[torch.FloatTensor of size 1]


-128.6861
[torch.FloatTensor of size 1]


-21.3697
[torch.

In [252]:
print selector.linear.weight.data
print selector_loss



selector.linear.weight.data += 1e-1*selector_loss

print selector.linear.weight


-16.1031 -16.0559 -16.0741  ...  -16.1102 -16.0916 -16.0917
-16.0661 -16.0424 -16.0574  ...  -16.0980 -16.1071 -16.1057
[torch.FloatTensor of size 2x712]


-40.1983
[torch.FloatTensor of size 1]

Parameter containing:
-20.1230 -20.0758 -20.0939  ...  -20.1301 -20.1114 -20.1116
-20.0859 -20.0623 -20.0773  ...  -20.1178 -20.1269 -20.1255
[torch.FloatTensor of size 2x712]



Hyperparamètres + apprentissage

# TODO

- step qui màj les paramètres (backward graph?)
          
- inputs via glove


- vérifier résultats après apprentissage


- faire des batchs


- implémenter skim rnn avec option lstm


- implémenter fonctions d'évaluations


In [255]:
def accuracy(indexes,preprocesser,embedder,selector,rnn):
    acc = 0.0
    for i in indexes:
        s,t =preprocesser.corpus[i], sentiment[i]
        input,target =  makeInputTarget(preprocesser, s, t)
        input_length = input.size()[0]
        
        for word in range(input_length):
            hidden = rnn.initHidden()
            x = input[word]
            output, p, hidden = rnn(Variable(x), hidden)  
        if output.exp().multinomial().data[0,0] == target.data[0]:
            acc += 1
            
    return 100*float(acc/len(indexes))